In [ ]:
# 2. Create a MobileNetV2 base model with reduced input size for Arduino compatibility
def create_person_detection_model():
    # Use MobileNetV2 as base model with smaller input size
    base_model = MobileNetV2(
        input_shape=(96, 96, 3),  # Smaller input size
        include_top=False,
        weights='imagenet'
    )

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add detection head
    x = base_model.output
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(4, (1, 1), padding='same')(x)  # 5 = [class, x, y, width, height]

    # Reshape output for bounding box prediction
    # shapes = tf.shape(base_model.input)
    # output = Reshape((-1, 5))(x)  # Reshape to [batch, num_boxes, 5]
    # output = Reshape((base_model.input_shape[1] * base_model.input_shape[2], 5))(x)  # Reshape to [batch, num_boxes, 5]
    output_shape = (base_model.output_shape[1] * base_model.output_shape[2], 4)
    output = Reshape(output_shape)(x)  # Reshape to [grid_cells, 5]

    model = Model(inputs=base_model.input, outputs=output)

    # #define pruning schedule
    # pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
    #     initial_sparsity=0.0,
    #     final_sparsity=0.5,
    #     begin_step=0,
    #     end_step=1000)
    
    # #prune the entire model
    # model = tfmot.sparsity.keras.prune_low_magnitude(model, pruning_schedule=pruning_schedule)
    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',  # Mean squared error for bounding box regression
        metrics=['accuracy']
    )

    return model